#### Q.1

In [22]:
from queue import PriorityQueue
import numpy as np
import copy

class Coord:
    def __init__(x, y):
        row = x
        col = y
        
#==========================Class to represent state of a configuration=================================
class State:
    def __init__(self, grid):
        self.grid = grid

    def __eq__(self, other):
        return np.array_equal(self.grid, other.grid)

    def __cmp__(self, other):
        return np.array_equal(self.grid, other.grid)

    # overriding hashing function
    def __hash__(self) :
        return hash(str(self.grid))
    
        
class QueueNode:
    def __init__(self, g, priority, parent, state, redBlock, action):
        self.g = g
        self.redBlock = redBlock
        self.priority = priority
        self.parent = parent
        self.state = state
        self.action = action
            
            #define compare function to compare the priority
#     def __cmp__(self, other):
#         return cmp(self.priority, other.priority)
    
    def __lt__(self, other):
        return self.priority < other.priority
    
#     def __eq__(self, other):
#         return (self.priority == other.priority)
            
n = int(input("Enter n: "))
grid = np.ones((n, n))
redBlock = list(map(int, (input("Enter Red block Position(i j): ").split())))
print(redBlock)
###initialize red block
grid[redBlock[0]][redBlock[1]] = 2

numEmpty = int(input("Enter number of empty blocks: "))
for i in range(numEmpty):
    b = list(map(int, (input("Enter "+str(i) + "empty block Position(i j): ").split())))
    grid[b[0]][b[1]] = 0
    
print(grid)

actionList = ['left', 'right', 'down', 'up']
actionToInd = {'left':0, 'right':1, 'down':2, 'up': 3}
actionToTuple = {'left':(0, -1), 'right':(0, 1), 'down':(1, 0), 'up': (-1, 0)}


Enter n: 4
Enter Red block Position(i j): 1 2
[1, 2]
Enter number of empty blocks: 2
Enter 0empty block Position(i j): 3 2
Enter 1empty block Position(i j): 2 3
[[1. 1. 1. 1.]
 [1. 1. 2. 1.]
 [1. 1. 1. 0.]
 [1. 1. 0. 1.]]


In [ ]:
import copy

def heuristic(n, redBlock):
#     print(n, redBlock)
    return (np.abs(redBlock[0] - 0) + np.abs(redBlock[1] - (n - 1)))

def reachedGoal(M):
    if(M[0, -1] == 2):
        return True

    return False

Q = PriorityQueue()
h = heuristic(grid, redBlock)
state = QueueNode(0, 0 + h, None, State(grid.copy()), tuple(redBlock), None)
Q.put(state)
explored = set()

def isValid(i, j):
    if(i > n-1 or i<0 or j > n - 1 or j<0):
        return False
    return True

def getNextState(M, i, j, action, redBlock):
    d = actionToTuple[action]
    x, y = i+d[0], j + d[1]
    if(not isValid(x, y)):
        return None, None
    
    if(M[x, y] == 0):
        return None, None
    else:
        if(redBlock == (x, y)):
            redBlock = i, j
        M[i, j], M[x, y] = M[x, y], M[i, j]
        return M, redBlock

destination = None
while(not Q.empty()):
    curr = Q.get()
    grid1 = curr.state.grid.copy()
    if reachedGoal(grid1):
        print("Reached")
        destination = curr
        break
    for i in range(n):
        for j in range(n):
            if(grid1[i, j] == 0):
                for action in actionList:
                    M, redBlock = getNextState(grid1.copy(), i, j, action, curr.redBlock)
                    if(M is None):
                        continue
                    if(not State(M) in explored):
                        explored.add(State(M))
                        h = heuristic(n, redBlock)
#                         print(h, g)
                        g = curr.g + 1
                        state = QueueNode(g, g + h, copy.copy(curr), State(M), redBlock, action)
                        Q.put(state)
        
#function to Print Path
def printPath(curr) :
    if(curr is None) : return 0
    cost = printPath(curr.parent)
    print(curr.action, "\n", curr.state.grid)
    return cost + 1

print("The path is as follows : \n")
cost = 0
cost= printPath(curr)
print("Cost = ", cost - 1)                     

In [ ]:
tuple(redBlock)

---

#### Q2

In [15]:
np.random.randn()

-0.25564592964390764

In [36]:
import time
import numpy as np
import copy

def f1(X):
    X = [x**2 for x in X]
    return np.sum(X)

def f2(X):
    X = [np.floor(x) for x in X]
    return np.sum(X)

def f3(X):
    X = [(i+1)*(x**4) for i, x in enumerate(X)]
    return np.sum(X) + np.random.randn()

population = []
Min = -2.04
Max = 2.04
populationSize = 50
epochs = 50
mutProb = 0.5
crossProb = 0.3
poolsize = 100

def value(x1):
    return f1(x1) + f2(x1) + f3(x1)

def fittest(x1, x2):
    if(value(x1) < value(x2)):
        return x1
    return x2

def fittestInPopulation():
    best1 = population[0]
    index = 0
    for x in population:
        if(value(x) > value(best1)):
            best1 = x
            index = x
    
    return best1, index

#Take a num and returns new number after mutation
def mutation(numList):
    newList = []
    for num in numList:
        binary = bin(int(num*100)).replace('0b','')

        if(num < 0):
            r_ind = np.random.randint(1, len(binary))
        else:
            r_ind = np.random.randint(len(binary))

        sList = list(binary)
        sList[r_ind] = str(1 - int(sList[r_ind]))
        binary = ''.join(sList)
        x = round(int(binary, 2)/100, 2)
        if(x < Min or x > Max):
            x = num
        if(np.random.rand() < 0.5):
            x = -x
        newList.append(x)
    return newList

#Cross Over
def crossOver(num1List, num2List):
    newList1 = []
    newList2 = []
    for (num1, num2) in zip(num1List, num2List):

        binary1 = bin(int(num1*100)).replace('0b','')
        binary2 = bin(int(num2*100)).replace('0b','')

        r1_ind = np.random.randint(min(len(binary1), len(binary2)))

        binary1_1 = binary1[:r1_ind] + binary2[r1_ind:]
        binary1_2 = binary2[:r1_ind] + binary1[r1_ind:]

        x1 = round(int(binary1_1, 2)/100, 2)
        x2 = round(int(binary1_2, 2)/100, 2)

        if(x1 < Min or x1 > Max or x2 < Min or x2 > Max):
            x1, x2 =  num1, num2

        if(np.random.rand() < 0.5):
            x1 = -x1
        if(np.random.rand() < 0.5):
            x2 = -x2
            
        newList1.append(x1)
        newList2.append(x2)

    return newList1, newList2

#Takes numList and return two best
def selection(numList):
    best1 = numList[0]
    for num in numList:
        if value(num)> value(best1):
            best1 = num
    
    return best1

#Decision making for crossover
def doCrossOver():
    return np.random.rand() > crossProb

#Decision making for mutation
def doMutation():
    return np.random.rand() > mutProb

def v(num1List, num2List):
#     print("s1 = ", s1, s2)
    if(doCrossOver()): 
        num1List, num2List = crossOver(num1List, num2List)
    
    offSpringList1 = []
    offSpringList2 = []
    for i in range(poolsize):
        #Pool1
        tempList1 = copy.copy(num1List)
        tempList2 = copy.copy(num2List)

        if(doMutation()):
            tempList1 = mutation(tempList1)
            offSpringList1.append(tempList1)

        #Pool2
        if(doMutation()):
            tempList2 = mutation(tempList2)
            offSpringList2.append(tempList2)

    #Selection
    offSpringList1.append(num1List)
    offSpringList1.append(num2List)
    offSpringList1+=offSpringList2   #Pool
    best1 = selection(offSpringList1)
        
    return best1

def getRandomNumber():
    x = np.random.rand()
    return round((4.08*x - 2.04), 2)

def initializePopulation():
    for i in range(populationSize):
        population.append([getRandomNumber() for _ in range(5)])

def geneticAlgorithm():
    global population
    initializePopulation()
    res, index = fittestInPopulation()
    bestVal = value(res)
    iter1 = 0
    while(iter1 < epochs):
        newPopulation = []
        for i in range(populationSize): 
            np.random.seed(np.random.randint(int(time.time())))
            parent1 = np.random.randint(populationSize)
            parent2 = np.random.randint(populationSize)
            bestOffSpring = v(population[parent1], population[parent2])
            if(value(bestOffSpring) > bestVal):
                bestVal = value(bestOffSpring)
                res = bestOffSpring
            newPopulation.append(bestOffSpring)
            
        iter1+=1
        if(iter1%10 == 0):
            print("Iter: ", iter1)
        population = newPopulation

    print("Found Goal in iteration = " , iter1, "Goal = ", res,  "Value = ", bestVal)

In [37]:
geneticAlgorithm()

Iter:  10
Iter:  20
Iter:  30
Iter:  40
Iter:  50
Found Goal in iteration =  50 Goal =  [2.04, 2.04, 2.04, 2.04, 2.04] Value =  291.8960475137854


In [242]:
int('010101',2)

21

---

#### Q3

In [87]:
import itertools

def poisson(lambda1, n):
    return (np.power(lambda1, n)/np.math.factorial(n))*(np.exp(-lambda1))

class State:
    def __init__(self, car1, car2, car3):
        self.car1 = car1
        self.car2 = car2
        self.car3 = car3
        
def normInfinity(currValue, optimalValue):
    maxDiff = 0
    for i in range(currValue.shape[0]):
        for j in range(currValue.shape[1]):
            maxDiff = max(maxDiff, abs(currValue[i, j] - optimalValue[i, j]))
    return maxDiff
        


requestLambda = [3, 2, 2]
returnLambda = [3, 1, 1]

gamma = 0.9
numIter = 50
tol = 0.1
Size = (20, 10, 10)
n1 = np.arange(20)
n2 = np.arange(10)
n3 = np.arange(10)

n4 = np.arange(-5, 5)

def isValidState(car1, car2, car3):
     return car1 >= 0 and car2 >=0 and car3 >=0
    
def normalize(car1, car2, car3):
    car1 = min(car1, 19)
    car2 = min(car2, 9)
    car3 = min(car3, 9)
    
    return car1, car2, car3

def getCost(mv1, mv2, mv3):
    cost = np.abs(mv1)*2
    return cost

def ValueIteration(V, Policy):
    for iter1 in range(numIter):
        print(iter1)
        #State value
        for i, j, k in itertools.product(n1, n2, n3):
            print(i, j, k)
            #Action
            maxReward = -10000
            for mv1, mv2, mv3 in itertools.product(n4, n4, n4):
                reward = 0.0
                if(mv1 + mv2 + mv3 != 0):
                    continue
                car1 = i + mv1
                car2 = j + mv2
                car3 = k + mv3
                
                reward-=getCost(mv1, mv2, mv3)
                
                #Request
                for b1, b2, b3 in itertools.product(n1, n2, n3):
                    pb1 = poisson(requestLambda[0], b1)
                    pb2 = poisson(requestLambda[1], b2)
                    pb3 = poisson(requestLambda[2], b3)
                    
                    car1-= b1
                    car2-= b2
                    car3-= b3
                    if(not isValidState(car1, car2, car3)):
                        continue
                    reqReward = 10*(b1 + b2 + b3)
                    
                    p1 = pb1*pb2*pb3
                    #Return 
                    for r1, r2, r3 in itertools.product(n1, n2, n3):
                        pr1 = poisson(returnLambda[0], r1)
                        pr2 = poisson(returnLambda[1], r2)
                        pr3 = poisson(returnLambda[2], r3)
                        
                        car1+=r1
                        car2+=r2
                        car3+=r3
                        
                        car1, car2, car3 = normalize(car1, car2, car3)
                        
                        p2 = pr1*pr2*pr3
                        reward+=p1*p2*(reqReward + gamma*V[car1, car2, car3])
                        
                maxReward = max(maxReward, reward)
            V[i, j, k] = maxReward  
        return V

In [82]:
Size = (20, 10, 10)
Value = np.zeros(Size)
Policy = np.zeros(Size)
V = ValueIteration(Value, Policy)

0
0 0 0


KeyboardInterrupt: 

In [ ]:
np.arange(-5, 6)

In [83]:
a = time.time()
for _ in range(280000000):
    j = 3
print(time.time()  - a)

KeyboardInterrupt: 

In [ ]:
a = np.ones((19))
c = np.ones((19))*2

In [ ]:
b = time.time()
np.sum(np.multiply(a,b))
print(time.time() - b)

In [58]:
a[:5]

array([1., 1., 1., 1., 1.])

In [52]:
c

array([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2.]])

In [53]:
a*c

array([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2.]])

In [84]:
JointProbLocation1 = np.zeros((20, 20, 20))
JointProbLocation2 = np.zeros((10, 10, 10))
JointProbLocation3 = np.zeros((10, 10, 10))

ProbRequestLocation1 = np.asarray([poisson(requestLambda[0], i) for i in range(20)])
ProbRequestLocation2 = np.asarray([poisson(requestLambda[1], i) for i in range(10)])
ProbRequestLocation3 = np.asarray([poisson(requestLambda[2], i) for i in range(10)])

ProbReturnLocation1 = np.asarray([poisson(returnLambda[0], i) for i in range(20)])
ProbReturnLocation2 = np.asarray([poisson(returnLambda[1], i) for i in range(10)])
ProbReturnLocation3 = np.asarray([poisson(returnLambda[2], i) for i in range(10)])

In [85]:
n1, n2, n3

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [99]:
for (req, retStart, retEnd) in itertools.product(n1, n1, n1):  #start, End inclusive
    if(req + 1 != retEnd - retStart + 1):
        continue
    JointProbLocation1[req, retStart, retEnd] = np.sum(
        ProbRequestLocation1[:req+1]*ProbReturnLocation1[retStart:retEnd+1])
    
for (req, retStart, retEnd) in itertools.product(n2, n2, n2):
    if(req + 1 != retEnd - retStart + 1):
        continue
    
    JointProbLocation2[req, retStart, retEnd] = np.sum(
        ProbRequestLocation2[:req+1]*ProbReturnLocation2[retStart:retEnd+1])
    
for (req, retStart, retEnd) in itertools.product(n3, n3, n3):
    if(req + 1 != retEnd - retStart + 1):
        continue
    
    JointProbLocation3[req, retStart, retEnd] = np.sum(
        ProbRequestLocation3[:req+1]*ProbReturnLocation3[retStart:retEnd+1])

In [102]:
(JointProbLocation1)

array([[[2.47875218e-03, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 7.43625653e-03, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 1.11543848e-02, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         8.99965629e-10, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 1.49994272e-10, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 2.36833060e-11]],

       [[0.00000000e+00, 2.47875218e-02, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 4.08994109e-02, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 

In [93]:
print(ProbRequestLocation1)
print(ProbReturnLocation1)
print(ProbRequestLocation1*ProbReturnLocation1)

[4.97870684e-02 1.49361205e-01 2.24041808e-01 2.24041808e-01
 1.68031356e-01 1.00818813e-01 5.04094067e-02 2.16040315e-02
 8.10151179e-03 2.70050393e-03 8.10151179e-04 2.20950322e-04
 5.52375804e-05 1.27471339e-05 2.73152870e-06 5.46305740e-07
 1.02432326e-07 1.80762929e-08 3.01271548e-09 4.75691918e-10]
[4.97870684e-02 1.49361205e-01 2.24041808e-01 2.24041808e-01
 1.68031356e-01 1.00818813e-01 5.04094067e-02 2.16040315e-02
 8.10151179e-03 2.70050393e-03 8.10151179e-04 2.20950322e-04
 5.52375804e-05 1.27471339e-05 2.73152870e-06 5.46305740e-07
 1.02432326e-07 1.80762929e-08 3.01271548e-09 4.75691918e-10]
[2.47875218e-03 2.23087696e-02 5.01947316e-02 5.01947316e-02
 2.82345365e-02 1.01644331e-02 2.54110829e-03 4.66734175e-04
 6.56344934e-05 7.29272148e-06 6.56344934e-07 4.88190446e-08
 3.05119029e-09 1.62489424e-10 7.46124905e-12 2.98449962e-13
 1.04923815e-14 3.26752364e-16 9.07645456e-18 2.26282801e-19]


In [90]:
np.sum(ProbRequestLocation1*ProbReturnLocation1)

0.16665743263981658

In [92]:
np.sum(ProbRequestLocation1)
np.sum(ProbReturnLocation1)

0.9999999999168558